Эта программа будет особенно полезна тем, кто учит языки. В ее арсенале есть несколько приемов, просто запустите код:

In [ ]:
 from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
from selenium import webdriver
import imgkit


def l_of_smth(tenses):
    lis=''
    for ten, s in enumerate(tenses,start = 1):
        lis = lis + str(ten) + ' ' +  str(s) + '\n'
    return lis

def gv(lst):
    v = ''
    if len(lst) > 1:
        del lst[0]
    for el in lst:
        v += el
    return v

def conjugate():
    language = input("Enter the language you are learning: ").lower() 
    if language not in list_of_languages:
        print("We can't help you with this language. Please, try again or choose another one.")
    else:
        word = input("Enter the word you want to conjugate: ")
        url = 'http://conjugator.reverso.net/conjugation-{}-verb-{}.html'.format(language, word)
        r = requests.get(url)
        page = BeautifulSoup(r.text, 'html.parser')
        blocks = [el for el in page('div') if el.get('class') == ["blue-box-wrap"]][0:(lang_tenses[language])] 
        tenses = [el.p.string for el in blocks]
        verbs = [[[v.string for v in i('i')] for i in el('li')] for el in blocks]
        if len(verbs) == 0:
            print('Please, try again and check the word you entered.')
        else:
            if tenses[8] == 'Présent ':
                tenses[8] = 'Présent conditionelle'
            pronouns = [i[0] for i in verbs[0]]
            all_forms = pd.DataFrame(verbs).transpose()
            all_forms.columns = tenses
            pronouns = dict(enumerate(pronouns))
            all_forms.rename(index = pronouns, inplace = True)
            all_forms = all_forms.applymap(lambda x: gv(x))
            n_tense = int(input("Enter the number of the tense you want to get from the following list: \n{}".format(str(l_of_smth(tenses)))))  
            if n_tense > lang_tenses[language]:
                print('Please, try again and choose a tense from those presented.')
            else:
                return  str(all_forms[[tenses[n_tense-1]]])
        
def definitions():
    browser = webdriver.Chrome('C:/Users/chromedriver.exe')
    word = input("Enter the pharase you want to know about: ")
    ref = 'https://en.oxforddictionaries.com/'
    browser.get(ref)
    text = browser.find_element_by_name('query')
    text.click()
    text.send_keys(word, u'\ue007')
    defs = browser.find_elements_by_class_name('ind')
    defs = [el.text for el in defs]
    if len(defs) != 0:
        i = 0
        print('{} may mean:'.format(word.title()))
        for el in defs:
            i += 1
            l = str(i) + '.'
            print(l, el)
    else:
        print("Please, try again. We didn't find exact matches for {}.".format(word.title()))        

def translate():
    
    u = 'https://tech.yandex.ru/translate/doc/dg/concepts/api-overview-docpage/#api-overview__languages'
    x = requests.get(u)
    pag = BeautifulSoup(x.text, 'html.parser')
    lng = pag.tbody.findAll('td') 
    lng = [el.string for el in lng][:-2]
    list_of_lang = []
    n = 0
    while len(lng) != 0:
        list_of_lang.append((lng[0], lng[1]))
        n += 1
        del lng[0]
        del lng[0]
    list_of_lang = pd.DataFrame(list_of_lang)
    col = ['language', 'code']
    list_of_lang.columns = col
    numbers = [str(el) for el in range(1, (n+1))]
    numbers = dict(enumerate(numbers))
    list_of_lang = list_of_lang.sort_values('language').reset_index(drop=True)
    list_of_lang.rename(index = numbers, inplace = True)
    list_of_lang.reset_index(level=0, inplace=True)
    #list_of_lang_1 = list_of_lang[:(len(list_of_lang)//2)]
    #list_of_lang_2 = list_of_lang[(len(list_of_lang)//2):]
    #list_of_lang = pd.concat([list_of_lang_1, list_of_lang_1], axis =1)
    lg_1 = input('Enter the number of the language you want to translate from: \n{}'.format(list_of_lang.head(300)))
    lg_1 = list_of_lang.at[lg_1, 'code']
    lg_2 = input('Enter the number of the language you want to translate to: \n{}'.format(list_of_lang.head(300)))
    lg_2 = list_of_lang.at[lg_2, 'code']
    word = input("Enter the pharase you want to translate: ")
    to_from = "{}-{}".format(lg_1, lg_2)
    params_ya = {"key": "trnsl.1.1.20190414T173949Z.c4761fa3df2b7db3.431a5104851cbf9757c441ce90932806ec38f95c", "text": word, "lang": to_from}
    ya_js = requests.get("https://translate.yandex.net/api/v1.5/tr.json/translate", params = params_ya).json()
    trans = ya_js['text'][0] 
    print(trans)

list_of_languages = ['french', "english"]
lang_tenses = {'french': 11, "english": 12}
lang_rus = {'french': 'французский', "english": "английский"}
options = ['Conjugate a word (only french and english are available at this moment)', 'See the definitions of an english word', 'Translate']
to_do = int(input("Enter the number of the option you want to try: \n{}".format(l_of_smth(options))))
if to_do == 1:
    print(conjugate())
if to_do == 2:
    definitions()
if to_do == 3:
    translate()

Enter the number of the option you want to try: 
1 Conjugate a word (only french and english are available at this moment)
2 See the definitions of an english word
3 Translate
3


In [6]:
import sys 
!"{sys.executable}" -m pip install vk_api
import sys 
!"{sys.executable}" -m pip install selenium
import sys 
!"{sys.executable}" -m pip install imgkit


На курсе макроэкономики мы столкнулись с необходимостью построить регресссионную линию, для того чтобы отчистить данные о ВВП от шумов и смочь анализировать бизнес циклы.
Поэтому кажется крайне полезным научиться строить линейную регрессию и искать параметры модели.
Решая поставленную задачу методом наименьших квадратов мы приходим к нормальному уравнению.
Тогда давайте рассмотрим функцию y=cos(x) + eps при x принадлежащем отрезку [0, 2pi], eps имеющем распределение N(0, 0.1). В качестве свойств случайной величины x рассмотрим x^0, x^1 .. x^n.
Даввайте тогда напишем функцию, которая бы по значению n и по размеру обучающей выборки делала строила бы аппроксимизацию функции многочленом.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import math




def train(samples, adj):
    adj += 1;
    sample = 628
    x_cos = np.arange(sample) / 100
    y_cos = np.cos(x_cos)
    plt2 = plt.plot(x_cos, y_cos)    

    x = [random.uniform(0, 2 * math.pi) for i in range(samples)]
    x = np.array([[x[j] ** i for i in range(adj)] for j in range(samples)])
    y = np.cos(x[:,1]) + np.random.normal(0, 0.1, samples)
    w = np.zeros((1, adj))
    w[0, :] = np.dot(np.linalg.inv(np.dot(x.T, x)), np.dot(x.T, y))
    plt.scatter(x[:,1], y)
    
    sample = 628
    x_linear = np.arange(sample) / 100
    x_adj = np.zeros((adj, sample))
    for i in range(adj):
        x_adj[i,:] = x_linear ** i
    y_linear = np.dot(w, x_adj).T

    plt1 = plt.plot(x_linear, y_linear, color='cyan')
    
    
    plt.show(plt1, plt2)

    return(w.tolist()[0])

train(20,3)

<Figure size 640x480 with 1 Axes>

[1.5400356528142263,
 -1.537215119694892,
 0.2930427002697793,
 -0.008877164844698526]